In [17]:
# Enhancing Customer Churn Prediction with Continuous Experiment Tracking

#!pip install -q optuna comet_ml
#!pip install wandb
import optuna
import comet_ml
from comet_ml import Experiment

In [18]:
#pip install kaggle

In [19]:
#!pip install wandb

In [20]:
#!pip3 install comet_ml

In [21]:
import wandb
from kaggle_secrets import UserSecretsClient
#from kaggle_web_client import kaggle_web_client

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, log_loss

from kaggle_secrets import UserSecretsClient

# Set display options to show all columns
pd.set_option('display.max_columns', None)

In [23]:
#pip install --user kaggle

In [24]:
#!pip install kaggle

In [25]:
!kaggle --Version

usage: kaggle [-h] [-v]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...
kaggle: error: the following arguments are required: command


In [26]:
#pip install torchvision 

In [27]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

experiment = Experiment(
  api_key="9nnLTrZkBIwrfeNd9Pr4lgEE9",
  project_name="general",
  workspace="salioudiedhiou"
)


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in 'C:\\Users\\sdiedhio\\Desktop\\Formation Data Scientist\\features_importance-main' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/salioudiedhiou/general/be698a3e379a48de96de698357c4cf3f



In [28]:
import random

from comet_ml import ConfusionMatrix, Experiment, init
from comet_ml.integration.pytorch import log_model

import numpy as np
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

hyper_params = {
    "sequence_length": 28,
    "input_size": 28,
    "hidden_size": 128,
    "num_layers": 3,
    "num_classes": 10,
    "batch_size": 100,
    "num_epochs": 2,
    "learning_rate": 0.02,
}

# Login to Comet if needed
init()


experiment = Experiment(project_name="comet-example-pytorch-rich-logging")

experiment.add_tag("pytorch")

# Log hyperparameters to Comet
experiment.log_parameters(hyper_params)

# MNIST Dataset
train_dataset = dsets.MNIST(
    root="./data/", train=True, transform=transforms.ToTensor(), download=True
)

test_dataset = dsets.MNIST(root="./data/", train=False, transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=hyper_params["batch_size"], shuffle=False
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=hyper_params["batch_size"], shuffle=False
)


# Log dataset sample images to Comet
num_samples = len(train_dataset)
for _ in range(10):
    value = random.randint(0, num_samples)
    tmp, _ = train_dataset[value]
    img = tmp.numpy()[0]
    experiment.log_image(img, name="groundtruth:{}".format(_))


# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial states
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))

        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))

        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out


rnn = RNN(
    hyper_params["input_size"],
    hyper_params["hidden_size"],
    hyper_params["num_layers"],
    hyper_params["num_classes"],
)

experiment.set_model_graph(str(rnn))

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=hyper_params["learning_rate"])


def train_index_to_example(index):
    tmp, _ = train_dataset[index]
    img = tmp.numpy()[0]
    data = experiment.log_image(img, name="train_%d.png" % index)

    if data is None:
        return None

    return {"sample": str(index), "assetId": data["imageId"]}


def test_index_to_example(index):
    tmp, _ = test_dataset[index]
    img = tmp.numpy()[0]
    data = experiment.log_image(img, name="test_%d.png" % index)

    if data is None:
        return None

    return {"sample": str(index), "assetId": data["imageId"]}


def onehot(i):
    v = [0] * 10
    v[i] = 1
    return v


# Make one to use repeatedly, to re-use examples where possible:
confusion_matrix = ConfusionMatrix(index_to_example_function=train_index_to_example)

# Train the Model
total_steps = len(train_dataset) // hyper_params["batch_size"]
with experiment.train():

    print("Logging weights as histogram (before training)...")
    # Log model weights
    weights = []
    for name in rnn.named_parameters():
        if "weight" in name[0]:
            weights.extend(name[1].detach().numpy().tolist())
    experiment.log_histogram_3d(weights, step=0)

    step = 0
    for epoch in range(hyper_params["num_epochs"]):
        experiment.log_current_epoch(epoch)
        correct = 0
        total = 0

        epoch_predictions = None
        epoch_targets = None

        for i, (images, labels) in enumerate(train_loader):
            images = Variable(
                images.view(
                    -1, hyper_params["sequence_length"], hyper_params["input_size"]
                )
            )
            labels = Variable(labels)

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = rnn(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Compute train accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += float((predicted == labels.data).sum())

            if epoch_predictions is not None:
                epoch_predictions = np.concatenate(
                    (epoch_predictions, outputs.data.numpy())
                )
            else:
                epoch_predictions = outputs.data.numpy()

            if epoch_targets is not None:
                epoch_targets = np.concatenate(
                    (epoch_targets, np.array([onehot(v) for v in labels]))
                )
            else:
                epoch_targets = np.array([onehot(v) for v in labels])

            # Log accuracy to Comet.ml
            experiment.log_metric("accuracy", correct / total, step=step)
            step += 1

            if (i + 1) % 100 == 0:
                print(
                    "Epoch [%d/%d], Step [%d/%d], Loss: %.4f"
                    % (
                        epoch + 1,
                        hyper_params["num_epochs"],
                        i + 1,
                        total_steps,
                        loss.data.item(),
                    )
                )

        # At end of epoch:
        print("Computing confusion matrix and uploading samples...")
        confusion_matrix.compute_matrix(epoch_targets, epoch_predictions)
        experiment.log_confusion_matrix(
            matrix=confusion_matrix,
            title="Train Confusion Matrix, Epoch #%s" % (epoch + 1,),
            file_name="train-confusion-matrix-%03d.json" % (epoch + 1),
        )

        print("Logging weights as histogram...")
        # Log model weights
        weights = []
        for name in rnn.named_parameters():
            if "weight" in name[0]:
                weights.extend(name[1].detach().numpy().tolist())
        experiment.log_histogram_3d(weights, step=epoch + 1)


with experiment.test():
    # Test the Model
    correct = 0
    total = 0

    test_predictions = None
    test_targets = None
    for images, labels in test_loader:
        images = Variable(
            images.view(-1, hyper_params["sequence_length"], hyper_params["input_size"])
        )
        outputs = rnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += float((predicted == labels).sum())

        if test_predictions is None:
            test_predictions = np.array([onehot(v) for v in labels])
        else:
            test_predictions = np.concatenate(
                (test_predictions, np.array([onehot(v) for v in labels]))
            )

        if test_targets is None:
            test_targets = outputs.data.numpy()
        else:
            test_targets = np.concatenate((test_targets, outputs.data.numpy()))

    experiment.log_confusion_matrix(
        test_targets,
        test_predictions,
        title="Test Confusion Matrix",
        file_name="test-confusion-matrix.json",
        index_to_example_function=test_index_to_example,
    )

    experiment.log_metric("accuracy", correct / total)
    print(
        "Test Accuracy of the model on the 10000 test images: %d %%"
        % (100 * correct / total)
    )

# Log the model to Comet for easy tracking and deployment
log_model(experiment, rnn, "Pytorch-Mnist-Rich")

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/salioudiedhiou/general/be698a3e379a48de96de698357c4cf3f
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: Not all initial data has been logged for experiment be698a3e379a48de96de698357c4cf3f, call Experiment.end() to ensure that all data to have been logged
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COME

Logging weights as histogram (before training)...
Epoch [1/2], Step [100/600], Loss: 2.3155
Epoch [1/2], Step [200/600], Loss: 2.3041
Epoch [1/2], Step [300/600], Loss: 2.3369
Epoch [1/2], Step [400/600], Loss: 2.3814
Epoch [1/2], Step [500/600], Loss: 2.3716
Epoch [1/2], Step [600/600], Loss: 2.3183
Computing confusion matrix and uploading samples...
Logging weights as histogram...
Epoch [2/2], Step [100/600], Loss: 2.3133
Epoch [2/2], Step [200/600], Loss: 2.3189
Epoch [2/2], Step [300/600], Loss: 2.3264
Epoch [2/2], Step [400/600], Loss: 2.3737
Epoch [2/2], Step [500/600], Loss: 2.3738
Epoch [2/2], Step [600/600], Loss: 2.3182
Computing confusion matrix and uploading samples...
Logging weights as histogram...
Test Accuracy of the model on the 10000 test images: 9 %


In [29]:
data=pd.read_csv("Telco-Customer-Churn.csv", index_col=0)

In [30]:
data.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [31]:
# 3. Preprocessing

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null 

In [33]:
data=data.drop("TotalCharges",axis=1)

In [34]:
### reccoder Churn en 1 et 0

In [35]:
churn_numeric = {'Yes':1, 'No':0}
data.Churn.replace(churn_numeric, inplace=True)

In [36]:
#data

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null 

In [38]:
categorical_features=data.select_dtypes("object")


In [39]:
# categorical_features=categorical_features.drop("customerID",axis=1)

In [40]:
numerical_features=data.select_dtypes(["float64","int64"])


In [41]:
categorical_features=pd.get_dummies(categorical_features).astype('int64')
categorical_features

,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,1,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
5575-GNVDE,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1
3668-QPYBK,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1
7795-CFOCW,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0
9237-HQITU,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1
2234-XADUH,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0
4801-JZAZL,1,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0


In [42]:
data=numerical_features.join(categorical_features)

In [43]:
#data

In [44]:
#data.isna().sum()

In [45]:
 #Encode categorical features, scale numerical features

encoder = OneHotEncoder(handle_unknown="ignore", sparse=True)
scaler = StandardScaler()

X_train, X_val, y_train, y_val = train_test_split(data.drop("Churn", axis=1), data["Churn"], test_size=0.2, random_state=42)


In [46]:
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [47]:
X_val_scaled

array([[-0.4377492 , -1.28460467, -1.33162874, ..., -0.52765585,
         1.40914569, -0.54274967],
       [-0.4377492 ,  0.35323794, -1.31667194, ..., -0.52765585,
        -0.70964983, -0.54274967],
       [-0.4377492 ,  0.80364466, -1.51277218, ..., -0.52765585,
        -0.70964983,  1.84247002],
       ...,
       [-0.4377492 , -0.62946762, -1.49449165, ..., -0.52765585,
        -0.70964983, -0.54274967],
       [-0.4377492 ,  1.49972776, -0.69513389, ...,  1.89517467,
        -0.70964983, -0.54274967],
       [-0.4377492 , -1.28460467, -1.11392424, ..., -0.52765585,
        -0.70964983,  1.84247002]])

In [48]:
# 4. Model Training
# 5. Hyperparameter Tuning & Ensemble Modeling

In [49]:
def objective(trial):
    # Define hyperparameter search space for individual models
    rf_params = {
        'n_estimators': trial.suggest_int('rf_n_estimators', 100, 300),
        'max_depth': trial.suggest_categorical('rf_max_depth', [None, 10, 20]),
        'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('rf_min_samples_leaf', 1, 4),
    }

    gb_params = {
        'n_estimators': trial.suggest_int('gb_n_estimators', 100, 300),
        'learning_rate': trial.suggest_float('gb_learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_categorical('gb_max_depth', [3, 4, 5]),
    }

    svm_params = {
        'C': trial.suggest_categorical('svm_C', [0.1, 1, 10]),
        'kernel': trial.suggest_categorical('svm_kernel', ['linear', 'rbf']),
    }

    # Create models with suggested hyperparameters
    rf = RandomForestClassifier(**rf_params)
    gb = GradientBoostingClassifier(**gb_params)
    svm = SVC(probability=True, **svm_params)

    # Train individual models
    rf.fit(X_train_scaled, y_train)
    gb.fit(X_train_scaled, y_train)
    svm.fit(X_train_scaled, y_train)

    # Evaluate individual models on validation data
    rf_predictions = rf.predict(X_val_scaled)
    gb_predictions = gb.predict(X_val_scaled)
    svm_predictions = svm.predict(X_val_scaled)

    # Calculate accuracy and ROC AUC for individual models
    rf_accuracy = accuracy_score(y_val, rf_predictions)
    gb_accuracy = accuracy_score(y_val, gb_predictions)
    svm_accuracy = accuracy_score(y_val, svm_predictions)

    rf_roc_auc = roc_auc_score(y_val, rf.predict_proba(X_val_scaled)[:, 1])
    gb_roc_auc = roc_auc_score(y_val, gb.predict_proba(X_val_scaled)[:, 1])
    svm_roc_auc = roc_auc_score(y_val, svm.predict_proba(X_val_scaled)[:, 1])

    # Create a stacking ensemble with trained models
    estimators = [
        ('random_forest', rf),
        ('gradient_boosting', gb),
        ('svm', svm)
    ]

    stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

    # Train the stacking ensemble
    stacking_classifier.fit(X_train_scaled, y_train)

    # Evaluate the stacking ensemble on validation data
    stacking_predictions = stacking_classifier.predict(X_val_scaled)
    stacking_accuracy = accuracy_score(y_val, stacking_predictions)
    stacking_roc_auc = roc_auc_score(y_val, stacking_classifier.predict_proba(X_val_scaled)[:, 1])

    # Log parameters and metrics to Comet ML
    experiment.log_parameters({
        'rf_n_estimators': rf_params['n_estimators'],
        'rf_max_depth': rf_params['max_depth'],
        'rf_min_samples_split': rf_params['min_samples_split'],
        'rf_min_samples_leaf': rf_params['min_samples_leaf'],
        'gb_n_estimators': gb_params['n_estimators'],
        'gb_learning_rate': gb_params['learning_rate'],
        'gb_max_depth': gb_params['max_depth'],
        'svm_C': svm_params['C'],
        'svm_kernel': svm_params['kernel']
    })

    experiment.log_metrics({
        'rf_accuracy': rf_accuracy,
        'gb_accuracy': gb_accuracy,
        'svm_accuracy': svm_accuracy,
        'rf_roc_auc': rf_roc_auc,
        'gb_roc_auc': gb_roc_auc,
        'svm_roc_auc': svm_roc_auc,
        'stacking_accuracy': stacking_accuracy,
        'stacking_roc_auc': stacking_roc_auc
    })

    # Return the negative accuracy as Optuna aims to minimize the objective
    return -stacking_accuracy

In [50]:
# 6. Optimization Results

In [51]:
from tabulate import tabulate

# Create and optimize the study
study = optuna.create_study(direction='minimize')  # Adjust direction based on your optimization goal
study.optimize(objective, n_trials=15)  # 100 You can adjust the number of trials

# Get the best hyperparameters and results
best_rf_params = study.best_params
best_accuracy = -study.best_value  # Convert back to positive accuracy

# Convert the dictionary to a list of key-value pairs for tabulation
param_table = [(key, value) for key, value in best_rf_params.items()]

# Display the best_rf_params table
best_rf_params = tabulate(param_table, headers=["Parameter", "Value"], tablefmt="grid")

print(f"Best RF Hyperparameters:\n{best_rf_params}")
print(f"Best Accuracy: {best_accuracy}")

[I 2024-02-01 16:01:08,603] A new study created in memory with name: no-name-cd04e286-632b-4c76-a3d8-267263fcd335
[I 2024-02-01 16:02:30,263] Trial 0 finished with value: -0.8140525195173882 and parameters: {'rf_n_estimators': 179, 'rf_max_depth': 20, 'rf_min_samples_split': 6, 'rf_min_samples_leaf': 4, 'gb_n_estimators': 149, 'gb_learning_rate': 0.1925348504791541, 'gb_max_depth': 3, 'svm_C': 1, 'svm_kernel': 'linear'}. Best is trial 0 with value: -0.8140525195173882.
[I 2024-02-01 16:03:41,018] Trial 1 finished with value: -0.8112136266855926 and parameters: {'rf_n_estimators': 108, 'rf_max_depth': None, 'rf_min_samples_split': 4, 'rf_min_samples_leaf': 1, 'gb_n_estimators': 179, 'gb_learning_rate': 0.04539521975711506, 'gb_max_depth': 3, 'svm_C': 0.1, 'svm_kernel': 'rbf'}. Best is trial 0 with value: -0.8140525195173882.
[I 2024-02-01 16:06:54,096] Trial 2 finished with value: -0.8126330731014905 and parameters: {'rf_n_estimators': 123, 'rf_max_depth': 20, 'rf_min_samples_split': 10

Best RF Hyperparameters:
+----------------------+----------------------+
| Parameter            | Value                |
+======================+======================+
| rf_n_estimators      | 240                  |
+----------------------+----------------------+
| rf_max_depth         | 10                   |
+----------------------+----------------------+
| rf_min_samples_split | 4                    |
+----------------------+----------------------+
| rf_min_samples_leaf  | 1                    |
+----------------------+----------------------+
| gb_n_estimators      | 279                  |
+----------------------+----------------------+
| gb_learning_rate     | 0.012719777241766289 |
+----------------------+----------------------+
| gb_max_depth         | 4                    |
+----------------------+----------------------+
| svm_C                | 0.1                  |
+----------------------+----------------------+
| svm_kernel           | linear               |
+--------------

In [52]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/salioudiedhiou/comet-example-pytorch-rich-logging/4a3e8941aa23438a85c95ebc61da7386
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     gb_accuracy [15]       : (0.7906316536550745, 0.8176011355571328)
COMET INFO:     gb_roc_auc [15]        : (0.8358529920192117, 0.8621230863187967)
COMET INFO:     rf_accuracy [15]       : (0.7970191625266146, 0.8119233498935415)
COMET INFO:     rf_roc_auc [15]        : (0.8400517043278437, 0.8611397207241711)
COMET INFO:     stacking_accuracy [15] : (0.8026969481902059, 0.8183108587650816)
COMET INFO:     stacking_roc_auc [15]  : (0.8556121709606964, 0.8645245686130405)
COMET INFO:     svm

COMET INFO:     min_samples_leaf                            : 1
COMET INFO:     min_samples_split                           : 2
COMET INFO:     min_weight_fraction_leaf                    : 0.0
COMET INFO:     multi_class                                 : auto
COMET INFO:     n_estimators                                : 243
COMET INFO:     n_iter_no_change                            : None
COMET INFO:     n_jobs                                      : None
COMET INFO:     num_classes                                 : 10
COMET INFO:     num_epochs                                  : 2
COMET INFO:     num_layers                                  : 3
COMET INFO:     oob_score                                   : False
COMET INFO:     passthrough                                 : False
COMET INFO:     penalty                                     : l2
COMET INFO:     probability                                 : True
COMET INFO:     random_forest                               : RandomForestClas

In [53]:
experiment.log_html('<a href="www.comet.com"> I love Comet</a>')